In [2]:
import os
from dotenv import load_dotenv
from typing import Annotated, TypedDict
import operator
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages

# Load API keys and set up tracing
load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGSMITH_PROJECT"] = "Intro to LangGraph"

# Define the State Schema
# 'add_messages' is a special helper that appends new messages to the list.
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]

# Define the Node
def call_llm(state: GraphState):
    llm = ChatOpenAI(model="gpt-4o")
    response = llm.invoke(state['messages'])
    # The 'add_messages' reducer automatically appends this response
    return {"messages": [response]}

# Build the Graph
workflow = StateGraph(GraphState)
workflow.add_node("llm", call_llm)
workflow.add_edge("llm", END)
workflow.set_entry_point("llm")

# Compile the graph
app = workflow.compile()

# --- Run the Graph with Message Trimming ---

# Here's a long list of messages to simulate a conversation
inputs = [
    HumanMessage(content="Hi! I'm Bob."),
    HumanMessage(content="I'm a builder."),
    HumanMessage(content="I'm looking for a new tool."),
    HumanMessage(content="I want to build a custom agent."),
    HumanMessage(content="What should I use?")
]

print("--- Running graph with trimmed message history ---")

# In the config, we can pass a function to transform the state for a specific node.
# Here, we're telling the "llm" node to only use the last 2 messages.
final_state = app.invoke({"messages": inputs}, config={"llm": {"messages": lambda x: x[-2:]}})

print("\n--- Final AI Response ---")
# The final answer is based only on the trimmed history
print(final_state['messages'][-1].content)

--- Running graph with trimmed message history ---

--- Final AI Response ---
Building a custom agent can be a rewarding project, and the tools you use will depend on the specific tasks you want the agent to perform. Here are some options and approaches you might consider:

1. **Programming Languages**:
   - **Python**: Known for its simplicity and vast array of libraries, Python is ideal for building agents, especially those that involve data processing, machine learning, or automation.
   - **JavaScript (Node.js)**: Ideal for real-time applications and web-based agents.
   - **Java**: Good for enterprise-level applications where performance and scalability are crucial.

2. **Frameworks and Libraries**:
   - **Dialogflow**: Useful for building conversational agents. It's backed by Google and offers integrations with various platforms.
   - **Rasa**: An open-source Python framework for building AI assistants that give you full control over your machine learning models and data.
   - **